In [1]:
import pandas as pd
import numpy as np

In [2]:
IMREP_df = pd.read_csv("../summary_data/75bp/IMREP/IMREP_TRB_merged_extracted_features.csv")
TRUST4_df = pd.read_csv("../summary_data/75bp/TRUST4/TRUST4_TRB_merged_extracted_features.csv")
MIXCR_df = pd.read_csv("../summary_data/75bp/MIXCR/MIXCR_TRB_merged_extracted_features.csv")
CATT_df = pd.read_csv("../summary_data/75bp/CATT/CATT_TRB_merged_extracted_features.csv")
TCR_df = pd.read_csv("../summary_data/original/TCR_Seq/TCR_merged_extracted_features.csv")

In [3]:
# Select rows with nReads greater than 1
IMREP_df = IMREP_df[IMREP_df['nReads'] > 1]
TRUST4_df = TRUST4_df[TRUST4_df['nReads'] > 1]
MIXCR_df = MIXCR_df[MIXCR_df['nReads'] > 1]
CATT_df = CATT_df[CATT_df['nReads'] > 1]
TCR_df = TCR_df[TCR_df['nReads'] > 1]

In [4]:
# Rename nReads and frequencies columns according to tool for proper merging
IMREP_df = IMREP_df.rename(columns={"nReads": "nReads_IMREP"})
TRUST4_df = TRUST4_df.rename(columns={"nReads": "nReads_TRUST4"})
MIXCR_df = MIXCR_df.rename(columns={"nReads": "nReads_MIXCR"})
CATT_df = CATT_df.rename(columns={"nReads": "nReads_CATT"})
TCR_df = TCR_df.rename(columns={"nReads": "nReads_TCR"})

In [5]:
# Select rows on PBMC samples
TCR_df = TCR_df[TCR_df['Sample'].isin(['sample01','sample02','sample03','sample04','sample05'])]

Complete dataframe across all samples and tools

In [6]:
# Merge dataframes based on two key combination: Sample and CDR3. Outer join ensures no data is lost for instances that do not have overlap
merge_IMREP_TRUST4 = pd.merge(IMREP_df, TRUST4_df, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_IMREP_TRUST4_MIXCR = pd.merge(MIXCR_df, merge_IMREP_TRUST4, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_IMREP_TRUST4_MIXCR_CATT = pd.merge(CATT_df, merge_IMREP_TRUST4_MIXCR, how='outer', on=['Sample', 'CDR3']).fillna(0)
merge_complete = pd.merge(TCR_df, merge_IMREP_TRUST4_MIXCR_CATT, how='outer', on=['Sample', 'CDR3']).fillna(0)

merge_complete

,Sample,CDR3,nReads_TCR,nReads_CATT,nReads_MIXCR,nReads_IMREP,nReads_TRUST4
0,sample01,CAWRGDTAQQPQHF,2.0,0.0,0.0,0.0,0.0
1,sample01,CASSRDSPETQYF,2.0,0.0,0.0,0.0,0.0
2,sample01,CASSYSGRALGTGELFF,2.0,0.0,0.0,0.0,0.0
3,sample01,CASSPDGGLRSPLHF,2.0,2.0,0.0,0.0,0.0
4,sample01,CASTPRGTVTSNQPQHF,4.0,3.0,0.0,4.0,2.0
...,...,...,...,...,...,...,...
58323,sample04,CSTHGTAGRTDTQYF,0.0,0.0,0.0,0.0,3.0
58324,sample04,CASSPHPDSNQPQHF,0.0,0.0,0.0,0.0,4.0
58325,sample04,CASSPGSSYEQYF,0.0,0.0,0.0,0.0,2.0
58326,sample04,CASSLEGLANGYTF,0.0,0.0,0.0,0.0,2.0


In [7]:
# Add the tissue type
merge_complete.loc[merge_complete['Sample']=='sample01','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample02','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample03','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample04','tissue'] = 'PBMC'
merge_complete.loc[merge_complete['Sample']=='sample05','tissue'] = 'PBMC'

In [8]:
# Add T cell rich or poor tissue type
merge_complete.loc[merge_complete['Sample']=='sample01','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample02','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample03','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample04','tissue_type'] = 'T_cell_rich'
merge_complete.loc[merge_complete['Sample']=='sample05','tissue_type'] = 'T_cell_rich'

In [9]:
# Calculate total number of reads in each sample
total_reads = merge_complete[['Sample','nReads_TCR','nReads_MIXCR','nReads_IMREP','nReads_TRUST4','nReads_CATT']].groupby('Sample').sum().rename(columns={'nReads_TCR':'total_reads_TCR','nReads_MIXCR':'total_reads_MIXCR','nReads_IMREP':'total_reads_IMREP','nReads_TRUST4':'total_reads_TRUST4','nReads_CATT':'total_reads_CATT'})
total_reads

,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,total_reads_CATT
Sample,,,,,
sample01,90577.0,147531.0,101887.0,41094.0,139121.0
sample02,87762.0,45348.0,47713.0,31027.0,52544.0
sample03,305953.0,52873.0,37415.0,13885.0,64584.0
sample04,104779.0,277.0,1135.0,44.0,2155.0
sample05,18617.0,48.0,88.0,0.0,870.0


In [10]:
# Merge dataframes 
merge = pd.merge(merge_complete, total_reads, how='outer', on=['Sample']).fillna(0)

# Calculate frequency of CDR3 reads with respect to CDR3s that occur more than once
merge['frequency_TCR'] = merge['nReads_TCR'] / (merge['total_reads_TCR'] * 1.0)
merge['frequency_MIXCR'] = merge['nReads_MIXCR'] / (merge['total_reads_MIXCR'] * 1.0)
merge['frequency_IMREP'] = merge['nReads_IMREP'] / (merge['total_reads_IMREP'] * 1.0)
merge['frequency_TRUST4'] = merge['nReads_TRUST4'] / (merge['total_reads_TRUST4'] * 1.0)
merge['frequency_CATT'] = merge['nReads_CATT'] / (merge['total_reads_CATT'] * 1.0) 
merge.fillna(0, inplace=True)
merge

,Sample,CDR3,nReads_TCR,nReads_CATT,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,total_reads_CATT,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,frequency_CATT
0,sample01,CAWRGDTAQQPQHF,2.0,0.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,147531.0,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000000
1,sample01,CASSRDSPETQYF,2.0,0.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,147531.0,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000000
2,sample01,CASSYSGRALGTGELFF,2.0,0.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,147531.0,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000000
3,sample01,CASSPDGGLRSPLHF,2.0,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,147531.0,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000014
4,sample01,CASTPRGTVTSNQPQHF,4.0,3.0,0.0,4.0,2.0,PBMC,T_cell_rich,90577.0,147531.0,101887.0,41094.0,139121.0,0.000044,0.0,0.000039,0.000049,0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58323,sample05,CASSQEPSGDYNEQFF,0.0,0.0,0.0,4.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,870.0,0.000000,0.0,0.045455,0.000000,0.000000
58324,sample05,CASSLHGGAGEQYF,0.0,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,870.0,0.000000,0.0,0.022727,0.000000,0.000000
58325,sample05,CASVRESSPKREINVKAFF,0.0,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,870.0,0.000000,0.0,0.022727,0.000000,0.000000
58326,sample05,CASSLEGRTTAETQYF,0.0,0.0,0.0,6.0,0.0,PBMC,T_cell_rich,18617.0,48.0,88.0,0.0,870.0,0.000000,0.0,0.068182,0.000000,0.000000


Differentiate low_SDI and high_SDI repertoire

In [11]:
# Calculate Shannon-Wiener index
diversity_TCR = merge[['Sample','tissue','tissue_type','CDR3','nReads_TCR','total_reads_TCR','frequency_TCR']]
diversity_TCR = diversity_TCR[diversity_TCR.nReads_TCR != 0]
clonotype_count_TCR = diversity_TCR.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_TCR')

diversity_TCR['shannon_index_TCR'] = -(diversity_TCR['frequency_TCR']*np.log(diversity_TCR['frequency_TCR']))
shannon_TCR = diversity_TCR.groupby(['Sample']).agg({'shannon_index_TCR':'sum'}).reset_index().rename(columns={'':"shannon_index_TCR"})
shannon_TCR = pd.merge(shannon_TCR, clonotype_count_TCR, on=['Sample'])

# Define low SDI sample as the shannon_index < 2, high SDI sample as the shannon_index >= 2
shannon_TCR['repertoire_type'] = ['low_SDI' if x < 2 else 'high_SDI' for x  in shannon_TCR['shannon_index_TCR']]
repertoire_type = shannon_TCR[['Sample','repertoire_type']]
shannon_TCR

,Sample,shannon_index_TCR,clonotype_count_TCR,repertoire_type
0,sample01,1.072675,3001,low_SDI
1,sample02,1.001202,3229,low_SDI
2,sample03,0.960516,7697,low_SDI
3,sample04,9.598323,26802,high_SDI
4,sample05,8.005115,5440,high_SDI


In [12]:
# Generate metadata 
metadata = pd.merge(merge, repertoire_type, how='outer', on=['Sample'])
metadata.loc[:,'class'] = metadata["tissue_type"] +"_"+ metadata["repertoire_type"]

metadata.to_csv('../summary_data/75bp/all_tools_TRB.csv', index=False)
metadata

,Sample,CDR3,nReads_TCR,nReads_CATT,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,...,total_reads_IMREP,total_reads_TRUST4,total_reads_CATT,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,frequency_CATT,repertoire_type,class
0,sample01,CAWRGDTAQQPQHF,2.0,0.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,...,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000000,low_SDI,T_cell_rich_low_SDI
1,sample01,CASSRDSPETQYF,2.0,0.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,...,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000000,low_SDI,T_cell_rich_low_SDI
2,sample01,CASSYSGRALGTGELFF,2.0,0.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,...,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000000,low_SDI,T_cell_rich_low_SDI
3,sample01,CASSPDGGLRSPLHF,2.0,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,...,101887.0,41094.0,139121.0,0.000022,0.0,0.000000,0.000000,0.000014,low_SDI,T_cell_rich_low_SDI
4,sample01,CASTPRGTVTSNQPQHF,4.0,3.0,0.0,4.0,2.0,PBMC,T_cell_rich,90577.0,...,101887.0,41094.0,139121.0,0.000044,0.0,0.000039,0.000049,0.000022,low_SDI,T_cell_rich_low_SDI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58323,sample05,CASSQEPSGDYNEQFF,0.0,0.0,0.0,4.0,0.0,PBMC,T_cell_rich,18617.0,...,88.0,0.0,870.0,0.000000,0.0,0.045455,0.000000,0.000000,high_SDI,T_cell_rich_high_SDI
58324,sample05,CASSLHGGAGEQYF,0.0,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,...,88.0,0.0,870.0,0.000000,0.0,0.022727,0.000000,0.000000,high_SDI,T_cell_rich_high_SDI
58325,sample05,CASVRESSPKREINVKAFF,0.0,0.0,0.0,2.0,0.0,PBMC,T_cell_rich,18617.0,...,88.0,0.0,870.0,0.000000,0.0,0.022727,0.000000,0.000000,high_SDI,T_cell_rich_high_SDI
58326,sample05,CASSLEGRTTAETQYF,0.0,0.0,0.0,6.0,0.0,PBMC,T_cell_rich,18617.0,...,88.0,0.0,870.0,0.000000,0.0,0.068182,0.000000,0.000000,high_SDI,T_cell_rich_high_SDI


Calculate Shannon-Wiener index and absolute error for each sample across different tools

In [13]:
# MIXCR
diversity_MIXCR = merge[['Sample','tissue','tissue_type','CDR3','nReads_MIXCR','total_reads_MIXCR','frequency_MIXCR']]
diversity_MIXCR = diversity_MIXCR[diversity_MIXCR.nReads_MIXCR != 0]
clonotype_count_MIXCR = diversity_MIXCR.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_MIXCR['shannon_index_tool'] = -(diversity_MIXCR['frequency_MIXCR']*np.log(diversity_MIXCR['frequency_MIXCR']))
shannon_MIXCR = diversity_MIXCR.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_MIXCR = pd.merge(shannon_MIXCR, clonotype_count_MIXCR, on=['Sample'])
shannon_MIXCR['tool'] = 'MIXCR'

# IMREP
diversity_IMREP = merge[['Sample','tissue','tissue_type','CDR3','nReads_IMREP','total_reads_IMREP','frequency_IMREP']]
diversity_IMREP = diversity_IMREP[diversity_IMREP.nReads_IMREP != 0]
clonotype_count_IMREP = diversity_IMREP.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_IMREP['shannon_index_tool'] = -(diversity_IMREP['frequency_IMREP']*np.log(diversity_IMREP['frequency_IMREP']))
shannon_IMREP = diversity_IMREP.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_IMREP = pd.merge(shannon_IMREP, clonotype_count_IMREP, on=['Sample'])
shannon_IMREP['tool'] = 'IMREP'

# TRUST4
diversity_TRUST4 = merge[['Sample','tissue','tissue_type','CDR3','nReads_TRUST4','total_reads_TRUST4','frequency_TRUST4']]
diversity_TRUST4 = diversity_TRUST4[diversity_TRUST4.nReads_TRUST4 != 0]
clonotype_count_TRUST4 = diversity_TRUST4.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_TRUST4['shannon_index_tool'] = -(diversity_TRUST4['frequency_TRUST4']*np.log(diversity_TRUST4['frequency_TRUST4']))
shannon_TRUST4 = diversity_TRUST4.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_TRUST4 = pd.merge(shannon_TRUST4, clonotype_count_TRUST4, on=['Sample'])
shannon_TRUST4['tool'] = 'TRUST4'

# CATT
diversity_CATT = merge[['Sample','tissue','tissue_type','CDR3','nReads_CATT','total_reads_CATT','frequency_CATT']]
diversity_CATT = diversity_CATT[diversity_CATT.nReads_CATT != 0]
clonotype_count_CATT = diversity_CATT.groupby(['Sample'], sort=False).size().reset_index(name='clonotype_count_tool')

diversity_CATT['shannon_index_tool'] = -(diversity_CATT['frequency_CATT']*np.log(diversity_CATT['frequency_CATT']))
shannon_CATT = diversity_CATT.groupby(['Sample']).agg({'shannon_index_tool':'sum'}).reset_index().rename(columns={'':"shannon_index_tool"})
shannon_CATT = pd.merge(shannon_CATT, clonotype_count_CATT, on=['Sample'])
shannon_CATT['tool'] = 'CATT'

diversity = pd.concat([shannon_MIXCR,shannon_IMREP,shannon_TRUST4,shannon_CATT])
diversity = pd.merge(diversity, shannon_TCR, how='outer', on=['Sample'])
diversity = diversity.fillna(0)
tissue_type = merge[['Sample','tissue','tissue_type']].drop_duplicates(keep='first')
diversity = pd.merge(diversity, tissue_type, how='inner', on=['Sample'])
diversity['SDI_absolute_error'] = np.abs(diversity['shannon_index_TCR'] - diversity['shannon_index_tool'])
diversity.loc[:,'class'] = diversity["tissue_type"] +"_"+ diversity["repertoire_type"]

# Calculate clonality
diversity['clonality_TCR'] = 1 - (diversity['shannon_index_TCR']/np.log(diversity['clonotype_count_TCR']))
diversity['clonality_tool'] = 1 - (diversity['shannon_index_tool']/np.log(diversity['clonotype_count_tool']))
diversity['clonality_absolute_error'] = np.abs(diversity['clonality_TCR'] - diversity['clonality_tool'])

diversity.to_csv('../summary_data/75bp/all_tools_TRB_diversity.csv', index=False)
diversity

,Sample,shannon_index_tool,clonotype_count_tool,tool,shannon_index_TCR,clonotype_count_TCR,repertoire_type,tissue,tissue_type,SDI_absolute_error,class,clonality_TCR,clonality_tool,clonality_absolute_error
0,sample01,0.505884,1513,MIXCR,1.072675,3001,low_SDI,PBMC,T_cell_rich,0.566791,T_cell_rich_low_SDI,0.866028,0.930908,0.064880
1,sample01,1.101218,1990,IMREP,1.072675,3001,low_SDI,PBMC,T_cell_rich,0.028543,T_cell_rich_low_SDI,0.866028,0.855025,0.011003
2,sample01,1.368813,1448,TRUST4,1.072675,3001,low_SDI,PBMC,T_cell_rich,0.296138,T_cell_rich_low_SDI,0.866028,0.811923,0.054105
3,sample01,0.933154,2230,CATT,1.072675,3001,low_SDI,PBMC,T_cell_rich,0.139520,T_cell_rich_low_SDI,0.866028,0.878965,0.012937
4,sample02,0.665529,1103,MIXCR,1.001202,3229,low_SDI,PBMC,T_cell_rich,0.335672,T_cell_rich_low_SDI,0.876088,0.905003,0.028915
5,sample02,2.230410,3482,IMREP,1.001202,3229,low_SDI,PBMC,T_cell_rich,1.229209,T_cell_rich_low_SDI,0.876088,0.726510,0.149578
6,sample02,1.892770,1694,TRUST4,1.001202,3229,low_SDI,PBMC,T_cell_rich,0.891568,T_cell_rich_low_SDI,0.876088,0.745419,0.130669
7,sample02,1.741744,2540,CATT,1.001202,3229,low_SDI,PBMC,T_cell_rich,0.740542,T_cell_rich_low_SDI,0.876088,0.777837,0.098251
8,sample03,0.457852,712,MIXCR,0.960516,7697,low_SDI,PBMC,T_cell_rich,0.502664,T_cell_rich_low_SDI,0.892663,0.930291,0.037629
9,sample03,1.502244,2022,IMREP,0.960516,7697,low_SDI,PBMC,T_cell_rich,0.541727,T_cell_rich_low_SDI,0.892663,0.802644,0.090019
